In [1]:
# Video: 25. - Model registry (https://www.youtube.com/watch?v=TKHU7HAvGH8&list=PL3MmuxUbc_hIUISrluw_A7wDSmfOhErJK)
# start: 12:47

#Purpose: access models via script
import pandas as pd
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [4]:
# previously list_experiments
client.search_experiments()

[<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/3', creation_time=1748308559447, experiment_id='3', last_update_time=1748308559447, lifecycle_stage='active', name='my-cool-experiment', tags={}>,
 <Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/2', creation_time=1747521064076, experiment_id='2', last_update_time=1747521064076, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1747521029154, experiment_id='1', last_update_time=1747521029154, lifecycle_stage='active', name='my-brand-new-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1747519090377, experiment_id='0', last_update_time=1747519090377, lifecycle_stage='active', name='Default', tags={}>]

In [ ]:
#client.create_experiment(name = "my-cool-experiment")

MlflowException: Experiment(name=my-cool-experiment) already exists. Error: (raised as a result of Query-invoked autoflush; consider using a session.no_autoflush block if this flush is occurring prematurely)
(sqlite3.IntegrityError) UNIQUE constraint failed: experiments.name
[SQL: INSERT INTO experiments (name, artifact_location, lifecycle_stage, creation_time, last_update_time) VALUES (?, ?, ?, ?, ?)]
[parameters: ('my-cool-experiment', None, 'active', 1749340517193, 1749340517193)]
(Background on this error at: https://sqlalche.me/e/20/gkpj)

In [5]:
from mlflow.entities import ViewType

# finding the best or applicable runs
runs = client.search_runs(
experiment_ids = '2',
filter_string='metrics.rmse < 6.31',
max_results= 5,
order_by = ['metrics.rmse ASC']
)

In [6]:
for run in runs:
    print(f'run_id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}')

run_id: d3b1d08055064ce9b55944d9df946a14, rmse: 6.3056
run_id: ed343f43a43e4e769e25f8fbb25cf66b, rmse: 6.3095


In [7]:
import mlflow

#set this or it will look for models in local folder
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

run_uri = 'ed343f43a43e4e769e25f8fbb25cf66b'
run_id = f'runs:/{run_uri}/model'

result = mlflow.register_model(run_id, 'nyc-taxi-regressor')

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
Created version '7' of model 'nyc-taxi-regressor'.


In [8]:
client.search_registered_models()

[<RegisteredModel: aliases={'challenger': 2, 'champion': 4, 'client-test': 4}, creation_timestamp=1748307595453, description='The NYC predictor for trip duration', last_updated_timestamp=1750723979849, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1750723979849, current_stage='None', description=None, last_updated_timestamp=1750723979849, name='nyc-taxi-regressor', run_id='ed343f43a43e4e769e25f8fbb25cf66b', run_link=None, source='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/2/ed343f43a43e4e769e25f8fbb25cf66b/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=7>], name='nyc-taxi-regressor', tags={}>,
 <RegisteredModel: aliases={}, creation_timestamp=1748309625810, description=None, last_updated_timestamp=1748309625822, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1748309625822, current_stage='None', description=None, last_updated_timestamp=1748309625822, name='nyc_taxi_regressor', run_id='c18de08d074a43819

In [9]:
model_name = 'nyc-taxi-regressor'
latest_versions = client.get_latest_versions(name=model_name)[0]


/tmp/ipykernel_4711/1744030578.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)[0]


In [10]:
#get_latest_versions is deprecated because of the staging so from here it deviates from the video

client.set_registered_model_alias(model_name, alias='champion', version = latest_versions.version)

In [11]:
# You can also use client to update metadata
from datetime import datetime

date = datetime.today()
version = latest_versions.version
alias = client.get_model_version(name = model_name, version=version).aliases[0]
client.update_model_version(
    name = model_name,
    version = latest_versions.version,
    description= f"The model version {version} was transitioned to {alias} on {date}"
)

<ModelVersion: aliases=['champion'], creation_timestamp=1750723979849, current_stage='None', description='The model version 7 was transitioned to champion on 2025-06-24 00:13:11.415540', last_updated_timestamp=1750723991429, name='nyc-taxi-regressor', run_id='ed343f43a43e4e769e25f8fbb25cf66b', run_link=None, source='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/2/ed343f43a43e4e769e25f8fbb25cf66b/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=7>

In [12]:
# moving challenger models to production and archiving champion models
# this section i'm basically updating myself since the code still uses stages

#from sklearn.metrics import mean_squared_error
from sklearn.metrics import root_mean_squared_error
import pandas as pd


def read_dataframe(filename):
    #df = pd.read_csv(filename)
    df = pd.read_parquet(filename)
    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)

#def test_model(name, stage, X_test, y_test):
def test_model(name, alias, X_test, y_test):
    #model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    model = mlflow.pyfunc.load_model(f"models:/{name}@{alias}")

    y_pred = model.predict(X_test)
    return {"rmse": root_mean_squared_error(y_test, y_pred, squared=False)}
    #return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [13]:
# Jan dataset for train, Feb for validation, now March for testing

df = read_dataframe("data/green_tripdata_2021-03.parquet")

In [23]:
# something went wrong here where none of the runs have artifacts for some reason, so I'm just picking a random one I already have for now

client.download_artifacts(run_id=run_uri, path='/preprocessor.b', dst_path = '.')

MlflowException: The following failures occurred while downloading one or more artifacts from /workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/2/ed343f43a43e4e769e25f8fbb25cf66b/artifacts:
##### File /preprocessor.b #####
Invalid path